In [1]:
from fastbook import *
from pathlib import Path
# Define the search queries for woman and man portraits
searches = ['woman pictures portrait', 'man pictures portrait']

# Set the path to the directory where the dataset will be stored
path = Path('dataset')

# Check if the dataset directory doesn't exist
if not path.exists():
    # Create the dataset directory if it doesn't exist
    path.mkdir(exist_ok=True)
    
    # Loop through the search queries
    for o in searches:
        # Create a subdirectory within the dataset directory for each search query
        x = o.split()[0]
        dest = (path/x)
        dest.mkdir(exist_ok=True)
        
        # Search and download images based on the current query
        results = search_images_ddg(o)
        download_images(dest, urls=results[:200])
        
        # Resize the downloaded images to a maximum size of 400 pixels
        resize_images(dest, max_size=400, dest=dest)

# Verify the integrity of the downloaded images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

# Define how the image data will be loaded and processed
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.4, seed=42),
    get_y=parent_label,
    item_tfms=Resize(192, method='squish')
).dataloaders(path)

# Display a batch of images from the data loaders
dls.show_batch()

# Create a CNN learner using the data loaders and the resnet18 architecture
learn = cnn_learner(dls, resnet18, metrics=error_rate)

# Fine-tune the pre-trained model on the dataset for 3 epochs
learn.fine_tune(3)

# Search and download a woman portrait image if it doesn't exist
urls = search_images_ddg('woman pictures portrait', max_images=1)
dest = Path('Woman.jpg')
if not dest.exists():
    download_url(urls[0], dest, show_progress=True)

# Search and download a man portrait image if it doesn't exist
destf = Path('Man.jpg')
if not destf.exists():
    download_url(search_images_ddg('man pictures portrait', max_images=1)[0], destf, show_progress=True)

# Use the trained model to predict the gender of the woman image
gender, _, probs = learn.predict(PILImage.create('Woman.jpg'))

# Print the predicted gender and its probability
print(f"This is a: {gender}.")
print(f"Probability it is a woman: {probs[1]:.4f}.")
print(f"Probability it is a man: {probs[1]:.4f}.")